# Анализ твитов - обработка текста

Мы будем строить классификатор, который будет разделять текст на позитивные и негативные высказывания. Для этого мы воспользуемся уже размеченной базой. Загрузим данные для анализа

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import Image, display
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt
import pylab as pl
import re
import codecs
import nltk
import pymorphy2
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

%pylab inline
pylab.rcParams['figure.figsize'] = (15,10)

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [4]:
! pip install pymorphy2


## Задание 1. Загрузка данных

Источник: https://drive.google.com/file/d/1wnj7wWvpCZiB8-kpFdV6WhAQSip-GSLV/view?usp=share_link

- Загрузите датафреймы positive.csv и negative.csv
- Объедините датафреймы с помощью pd.concat() в один датафрейм;
- Убедитесь, что в новом датафрейме индексация сквозная и без повторов;
- Переименуйте столбцы датафрейма;
```
df.rename(columns = {0 : 'id', 1 : 'date', 2: 'name', 3: 'text', 4: 'positive', 5: 'rep', 6: 'rtv', 7: 'fav',
    8: 'total_count', 9: 'fol', 10: 'friends', 11: 'list_count'}, inplace = True)
```
- Выведите информацию об общем количестве полученных твитов, сколько из них негативных, сколько позитивных.

In [35]:
positive = pd.read_csv('positive.csv', sep=';', header=None, encoding_errors='ignore')
positive.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [36]:
negative = pd.read_csv('negative.csv', sep=';', header=None, encoding_errors='ignore')
negative.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [37]:
df = pd.concat([positive, negative], ignore_index=True)

# переименовываем колонки
df.rename(columns={
    0 : 'id', 
    1 : 'date', 
    2 : 'name', 
    3 : 'text', 
    4 : 'positive', 
    5 : 'rep', 
    6 : 'rtv', 
    7 : 'fav',
    8 : 'total_count', 
    9 : 'fol' , 
    10: 'friends', 
    11: 'list_count'
}, inplace=True)

df.head()

,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [38]:
df['text']

0         @first_timee хоть я и школота, но поверь, у на...
1         Да, все-таки он немного похож на него. Но мой ...
2         RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3         RT @digger2912: "Кто то в углу сидит и погибае...
4         @irina_dyshkant Вот что значит страшилка :D\nН...
                                ...                        
226829    Но не каждый хочет что то исправлять:( http://...
226830    скучаю так :-( только @taaannyaaa вправляет мо...
226831            Вот и в школу, в говно это идти уже надо(
226832    RT @_Them__: @LisaBeroud Тауриэль, не грусти :...
226833    Такси везет меня на работу. Раздумываю приплат...
Name: text, Length: 226834, dtype: object

In [10]:
print(f"Общее количество твитов {df.shape[0]}")
print(f"Позитивные твиты {len(df[df['positive']== 1])}")
print(f"Негативные твиты {len(df[df['positive']== -1])}")

Общее количество твитов 226834
Позитивные твиты 114911
Негативные твиты 111923


## Задание 2. Очистка данных

Произведите очистку данных. По результатам очистки выведите на экран следующую информацию:
- Общее количество слов перед удалением слов, встречающихся 1 раз;
- Количество слов, встречающихся 1 раз;
- Итоговое количество слов;
- Количество пустых твитов;
- Из них позитивных твитов;
- Количество твитов после удаления пустых.


In [40]:
#1 приведем все к нижнему регистру
df['text'] = df['text'].astype(str).str.lower()
df['text'].head()

0    @first_timee хоть я и школота, но поверь, у на...
1    да, все-таки он немного похож на него. но мой ...
2    rt @katiacheh: ну ты идиотка) я испугалась за ...
3    rt @digger2912: "кто то в углу сидит и погибае...
4    @irina_dyshkant вот что значит страшилка :d\nн...
Name: text, dtype: object

In [41]:
# 2. Оставьте в тексте только русские слова, удалив числа, знаки препинания, специальные символы и слова написанные латиницу

df['text'] = df['text'].str.replace(r"[^А-Яа-я]", " ", regex=True)
df['text']

0                      хоть я и школота  но поверь  у на...
1         да  все таки он немного похож на него  но мой ...
2                        ну ты идиотка  я испугалась за ...
3                          кто то в углу сидит и погибае...
4                         вот что значит страшилка    но...
                                ...                        
226829    но не каждый хочет что то исправлять          ...
226830    скучаю так     только             вправляет мо...
226831            вот и в школу  в говно это идти уже надо 
226832                             тауриэль  не грусти  ...
226833    такси везет меня на работу  раздумываю приплат...
Name: text, Length: 226834, dtype: object

In [13]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/polina/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [45]:
df['text'].apply(type).value_counts()

text
<class 'list'>    226834
Name: count, dtype: int64

In [46]:
# 0. Превращаем списки обратно в строки
df['text'] = df['text'].apply(
    lambda x: ' '.join(x) if isinstance(x, list) else str(x)
)

In [48]:
# 1. Приводим к нижнему регистру (можно ещё раз – это норм)
df['text'] = df['text'].astype(str).str.lower()
df['text']

0         хоть я и школота но поверь у нас то же самое о...
1         да все таки он немного похож на него но мой ма...
2                        ну ты идиотка я испугалась за тебя
3         кто то в углу сидит и погибает от голода а мы ...
4         вот что значит страшилка но блин посмотрев все...
                                ...                        
226829                 но не каждый хочет что то исправлять
226830    скучаю так только вправляет мозги но я все рав...
226831              вот и в школу в говно это идти уже надо
226832                             тауриэль не грусти обнял
226833    такси везет меня на работу раздумываю приплати...
Name: text, Length: 226834, dtype: object

In [49]:
# 3. Разбейте тексты на слова с помощью word_tokenize
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')  # если ещё не скачивала в этом окружении

df['tokens'] = df['text'].apply(word_tokenize)

df[['text', 'tokens']].head()

[nltk_data] Downloading package punkt to /Users/polina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,text,tokens
0,хоть я и школота но поверь у нас то же самое о...,"[хоть, я, и, школота, но, поверь, у, нас, то, ..."
1,да все таки он немного похож на него но мой ма...,"[да, все, таки, он, немного, похож, на, него, ..."
2,ну ты идиотка я испугалась за тебя,"[ну, ты, идиотка, я, испугалась, за, тебя]"
3,кто то в углу сидит и погибает от голода а мы ...,"[кто, то, в, углу, сидит, и, погибает, от, гол..."
4,вот что значит страшилка но блин посмотрев все...,"[вот, что, значит, страшилка, но, блин, посмот..."


In [19]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/polina/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [51]:
# 4. Удалите стоп-слова

from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
len(russian_stopwords)

151

In [59]:
# удаляем стоп-слова

def remove_stopwords(tokens):
    return [token for token in tokens if token not in russian_stopwords]

df['tokens'] = df['tokens'].apply(remove_stopwords)
df[['tokens']].head()

,tokens
0,"[школот, повер, обществ, предмет, тип]"
1,"[немн, похож, мальчик, равн]"
2,"[идиотк, испуга]"
3,"[угл, сид, погиба, голод, ещ, порц, взял, хот,..."
4,"[знач, страшилк, блин, посмотр, част, ощущен, ..."


In [22]:
pip install pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 5.7 MB/s  0:00:01m 5.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pymorphy3]
Note: you may need to restart the kernel to use updated packages.


In [54]:
# 5. Проведите лемматизацию

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

def stemming(tokens):
    return [stemmer.stem(word) for word in tokens]

df['tokens'] = df['tokens'].apply(stemming)
df[['tokens']].head()

,tokens
0,"[школот, повер, сам, обществ, профилир, предме..."
1,"[так, немн, похож, мальчик, равн]"
2,"[идиотк, испуга]"
3,"[угл, сид, погиба, голод, ещ, порц, взял, хот,..."
4,"[знач, страшилк, блин, посмотр, част, создаст,..."


In [58]:
# 6. Удалите все слова, которые встречаются только 1 раз

from collections import Counter

# считаем частоту слов во всех токенах
all_words = [word for tokens in df['tokens'] for word in tokens]
word_counts = Counter(all_words)

# фильтруем редкие слова
df['tokens'] = df['tokens'].apply(
    lambda tokens: [word for word in tokens if word_counts[word] > 1]
)

df[['tokens']].head()

,tokens
0,"[школот, повер, сам, обществ, предмет, тип]"
1,"[так, немн, похож, мальчик, равн]"
2,"[идиотк, испуга]"
3,"[угл, сид, погиба, голод, ещ, порц, взял, хот,..."
4,"[знач, страшилк, блин, посмотр, част, ощущен, ..."


In [60]:
# 7. Удалите строчки с пустыми твитами

df = df[df['tokens'].apply(len) > 0]
df.head()

,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count,tokens
0,408906692374446080,1386325927,pleease_shut_up,"[х, т, ь, , , , ш, л, т, , н, , п, е, р, ...",1,0,0,0,7569,62,61,0,"[школот, повер, обществ, предмет, тип]"
1,408906692693221377,1386325927,alinakirpicheva,"[д, , е, , т, , н, , н, е, м, н, г, , п, ...",1,0,0,0,11825,59,31,2,"[немн, похож, мальчик, равн]"
2,408906695083954177,1386325927,EvgeshaRe,"[н, , т, ы, , д, т, , , п, г, л, ь, , з, ...",1,0,1,0,1273,26,27,0,"[идиотк, испуга]"
3,408906695356973056,1386325927,ikonnikova_21,"[т, , т, , , г, л, , д, т, , , п, г, б, ...",1,0,1,0,1549,19,17,0,"[угл, сид, погиба, голод, ещ, порц, взял, хот,..."
4,408906761416867842,1386325943,JumpyAlex,"[т, , ч, т, , з, н, ч, т, , т, р, ш, л, , ...",1,0,0,0,597,16,23,1,"[знач, страшилк, блин, посмотр, част, ощущен, ..."


In [62]:
# 8. Какое количество твитов осталось. Сколько из них позитивных, сколько негативных?

total_tweets = len(df)
print(f"Общее количество твитов: {total_tweets}")

# Количество позитивных (1) и негативных (-1) твитов
positive_count = len(df[df['positive'] == 1])
negative_count = len(df[df['positive'] == -1])

print(f"Позитивных твитов: {positive_count}")
print(f"Негативных твитов: {negative_count}")

Общее количество твитов: 225901
Позитивных твитов: 114493
Негативных твитов: 111408


In [63]:
df.to_csv('cleaned_tweets.csv', index=False, encoding='utf-8')

## Задание 3. Классика ML

1. Кодировать данные методом мешка слов.
2. Построить классификатор на основе логистической регрессии
3. Построить классификатор на основе случайного леса

In [64]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.text, df.positive, test_size=0.3, random_state=42)

len(y_train), y_train.sum(), len(y_test), y_test.sum()

(158130, np.int64(2440), 67771, np.int64(645))

In [66]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# 1. Собираем чистый текст из токенов
# (предполагаем, что df['tokens'] уже очищены: стоп-слова, стемминг и т.д.)
df['text_clean'] = df['tokens'].apply(lambda tokens: ' '.join(tokens))

# 2. Делим на train / test
X_train, X_test, y_train, y_test = train_test_split(
    df['text_clean'],
    df['positive'],
    test_size=0.3,
    random_state=42,
    stratify=df['positive']
)

# 3. Строим мешок слов
bow = CountVectorizer(
    max_features=30000,
    ngram_range=(1, 2),
    min_df=2
)

X_train_bow = bow.fit_transform(X_train)
X_test_bow  = bow.transform(X_test)

print(X_train_bow.shape)
print(X_test_bow.shape)

feature_sample = bow.get_feature_names_out()[:25]
print("Примеры признаков:", feature_sample)

(158130, 30000)
(67771, 30000)
Примеры признаков: ['аа' 'аа блин' 'аа вс' 'аа дума' 'аа заб' 'аа завтр' 'аа наш' 'аа понятн'
 'аа поч' 'аа прост' 'аа страшн' 'аа хоч' 'аа эт' 'ааа' 'ааа блин'
 'ааа завтр' 'ааа хоч' 'ааа эт' 'аааа' 'аааа хоч' 'аааа эт' 'ааааа'
 'аааааа' 'ааааааа' 'аааааааа']


In [67]:
# 2. Логистическая регрессия по мешку слов
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score

logreg = LogisticRegression(
    solver='liblinear',
    class_weight='balanced',
    random_state=42,
    max_iter=500
)

logreg.fit(X_train_bow, y_train)
y_pred_lr = logreg.predict(X_test_bow)

acc_lr = accuracy_score(y_test, y_pred_lr)
f1_lr = f1_score(y_test, y_pred_lr, pos_label=1)

print(f"[LogReg] Accuracy: {acc_lr:.3f} | F1(positive=1): {f1_lr:.3f}")
print(classification_report(y_test, y_pred_lr, digits=3))


[LogReg] Accuracy: 0.717 | F1(positive=1): 0.721
              precision    recall  f1-score   support

          -1      0.714     0.713     0.713     33423
           1      0.721     0.722     0.721     34348

    accuracy                          0.717     67771
   macro avg      0.717     0.717     0.717     67771
weighted avg      0.717     0.717     0.717     67771



In [ ]:
# 3. Случайный лес по тем же признакам
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    n_jobs=-1,
    random_state=42,
    class_weight='balanced_subsample'
)

rf.fit(X_train_bow, y_train)
y_pred_rf = rf.predict(X_test_bow)

acc_rf = accuracy_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf, pos_label=1)

print(f"[RandomForest] Accuracy: {acc_rf:.3f} | F1(positive=1): {f1_rf:.3f}")
print(classification_report(y_test, y_pred_rf, digits=3))


In [70]:
print(f"[LogReg]       Accuracy: {acc_lr:.3f} | F1: {f1_lr:.3f}")
print(f"[RandomForest] Accuracy: {acc_rf:.3f} | F1: {f1_rf:.3f}")

[LogReg]       Accuracy: 0.717 | F1: 0.721
[RandomForest] Accuracy: 0.705 | F1: 0.696
